In [71]:
import pandas as pd
import numpy as np
import os 
from pathlib import Path

In [72]:
#changing directory to automatically retrieve the raw data

directory1 = Path.home() / 'Desktop' / 'epp_repo' / 'eppfinalproject_moritzraykowski' / 'src' / 'epp_final_project_MoritzRaykowski' / 'data'

In [73]:
os.chdir(directory1)

In [74]:
df = pd.read_excel('surveydataraw.xls', sheet_name='Sheet1')

In [75]:
# replacing unclear survey answers with missings
replaced_strings = ['unknown', 'no ans', 'no value', 'not fixed']
for string in replaced_strings:
    df.replace(string, np.nan, inplace=True )

In [76]:
# the survey has  multiple choice questions leading to categorial and ordinal variables which I encode using a map
map = {'Nein': 0, 'Ja': 1}
df = df.assign(
    partymember_encoded = df['partymember'].map(map),
    disabled_encoded = df['disabled'].map(map),
    reelection_coming_encoded = df['reelection_coming'].map(map),
    council_terms_encoded = df['council_terms'].map(map),
    council_existence_encoded = df['council_existence'].map(map),
    electoral_terms_encoded = df['electoral_terms'].map(map),
    existence_representative_encoded = df['existence_representative'].map(map),
)

In [77]:
# these variables have missings and are of data type object when imported so I coerce them to be numeric
object_cols = ['date_position_established', 'budget', 'duration_terms_in_years', 'council_term_length', 'date_council_established', 'disability_degree']
for col in object_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [78]:
# for more complex ordinal variables I use a map that gives numeric equivalents
ranks = {'Hauptschulabschluss': 1, 'Realschulabschluss': 2, 'Gymnasialabschluss': 3, 'Bachelor': 4, 'Master': 5, 'Doktor/PhD': 6,'Professur': 7, 'Kommissarisch': 1, 'Ehrenamtlich': 2,  'Nebenamtlich': 3, 'Hauptamtlich': 4, 'Chistlich Soziale Union in Bayern (CSU)': 1, 'Christlich Demokratische Union Deutschlands (CDU)': 2, 'Freie Wähler': 3, 'Sozialdemokratische Partei Deutschlands (SPD)': 4, 'Bündnis 90/Die Grünen': 5, 'Die Linke': 6}
df['voluntary_full_position'] = df['voluntary_full_position'].map(ranks)
df['education'] = df['education'].map(ranks)
df['partyname'] = df['partyname'].map(ranks)


In [79]:
#I create variables that will be used to differentiate experimental groups in the analysis part under different model specifications
df['Treatment_binary'] = df['Treatment_ID'].apply(lambda x: 1 if x == 1 or x == 2 else 0)
df['Treatment_full'] = df['Treatment_ID'].apply(lambda x: 1 if x == 1 else 0)
df['Treatment_attrition'] = df['Treatment_ID'].apply(lambda x: 1 if x == 2 else 0)

df['ControlvsAttrition'] = np.where(df['Treatment_ID'] == 0, 0, np.where(df['Treatment_ID'] == 2, 1, np.nan))
df['ControlvsFull'] = np.where(df['Treatment_ID'] == 0, 0, np.where(df['Treatment_ID'] == 1, 1, np.nan))
df['select'] = np.where(df['ControlvsFull'] == 0, 1, np.where(df['ControlvsFull'] == 1, 1, np.nan))


In [80]:
#switching to the directory where I safe the cleaned data
directory2 = Path.home() / 'Desktop' / 'epp_repo' / 'eppfinalproject_moritzraykowski' / 'src' / 'epp_final_project_MoritzRaykowski' / 'data_cleaned'
os.chdir(directory2)
df.to_excel("surveydatawpython.xlsx", index=False)